# Autism Face Classification — Colab Runner (Minimal)

This notebook is a minimal runner:
- Clones the GitHub repo
- Installs dependencies
- (Optionally) extracts the dataset
- Runs training and evaluation via **scripts/**

No training code lives here; everything runs through the repo pipeline.


In [ ]:
# ====== EDIT THESE 2 LINES ONLY ======
REPO_URL = "https://github.com/mVefa/autism-face-classification.git"
BRANCH   = "main"   # or "master"
# ====================================

import os, sys, subprocess, textwrap, pathlib
print("Repo:", REPO_URL)


In [ ]:
# Optional: mount Google Drive (if your dataset is in Drive)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# If you created requirements.txt, install it:
!pip -q install -r requirements.txt

# Extra safety (some Colab images need these):
!pip -q install opencv-python albumentations scikit-learn matplotlib
print("✅ Dependencies installed.")


## Dataset Setup

Expected dataset structure (physical k-fold):

data/autism_unified_kfold/
  fold_1/
    train/<class_name>/*.jpg
    val/<class_name>/*.jpg
  fold_2/...
  ...


In [ ]:
import os, zipfile, pathlib

# ===== EDIT THIS if you have a dataset zip in Drive =====
DATA_ZIP_PATH = "/content/drive/MyDrive/autism_unified_kfold.zip"
# =======================================================

TARGET_DIR = "/content/repo/data"

os.makedirs(TARGET_DIR, exist_ok=True)

if os.path.exists(DATA_ZIP_PATH):
    print("Extracting:", DATA_ZIP_PATH)
    with zipfile.ZipFile(DATA_ZIP_PATH, 'r') as z:
        z.extractall(TARGET_DIR)
    print("✅ Extracted to:", TARGET_DIR)
else:
    print("⚠️ Zip not found. If you already have the dataset folder, ignore this cell.")
    print("Expected dataset root:", os.path.join(TARGET_DIR, "autism_unified_kfold"))


In [ ]:
import os, glob

DATA_DIR = "/content/repo/data/autism_unified_kfold"

print("DATA_DIR:", DATA_DIR)
print("Exists:", os.path.exists(DATA_DIR))

# quick sanity check: list fold_1/train classes
fold1_train = os.path.join(DATA_DIR, "fold_1", "train")
print("fold_1/train exists:", os.path.exists(fold1_train))
if os.path.exists(fold1_train):
    print("Classes:", os.listdir(fold1_train))
    sample_imgs = glob.glob(os.path.join(fold1_train, "*", "*.jpg"))[:5]
    print("Sample images:", sample_imgs[:3])


## Train (K-Fold)

This will run the full physical K-Fold training and save:
- checkpoints under `saved_models/`
- logs/metrics under `outputs/`


In [ ]:
!python scripts/train.py \
  --data_dir ./data/autism_unified_kfold \
  --model resnet50 \
  --epochs 30 \
  --batch 32 \
  --img_size 224 \
  --lr 1e-4 \
  --wd 1e-4 \
  --folds 5 \
  --seed 42


## Evaluate (Example)

Evaluates a single fold checkpoint on fold_1/val and exports:
- outputs/eval_fold1/metrics.json
- outputs/eval_fold1/confusion_matrix.png


In [ ]:
# Example: evaluate fold_1 best checkpoint
!python scripts/eval.py \
  --data ./data/autism_unified_kfold/fold_1/val \
  --model resnet50 \
  --weights ./saved_models/resnet50_fold1.pth \
  --out ./outputs/eval_fold1

!ls -la ./outputs/eval_fold1


In [ ]:
# Optional: zip outputs + checkpoints to Drive
import os, shutil

OUT_ZIP = "/content/drive/MyDrive/autism_face_project_outputs.zip"

if os.path.exists("/content/drive"):
    if os.path.exists(OUT_ZIP):
        os.remove(OUT_ZIP)
    shutil.make_archive(OUT_ZIP.replace(".zip",""), 'zip', root_dir="/content/repo", base_dir="outputs")
    shutil.make_archive("/content/drive/MyDrive/autism_face_project_checkpoints", 'zip', root_dir="/content/repo", base_dir="saved_models")
    print("✅ Saved to Drive:")
    print(" -", OUT_ZIP)
    print(" - /content/drive/MyDrive/autism_face_project_checkpoints.zip")
else:
    print("Drive not mounted. Skip.")
